In [1]:
# the cards that are highlighted have the following class (in hand)
# class = card zoomable highlight handy
# the cards in hand that are not highlighted are as follows:
# class = card zoomable handy
# cards on board that are not highlighted
# card zoomable mappy
# cards on board controlled by opponent
# card zoomable mappy opponent-controlled
# cards on board you control that are highlighted
# card zoomable highlight mappy
# choice of taking an action (e.g. attacking, discarding a card, summoning a card, etc.)
# action-choice
# this shows up when attacking an enemy, 
# class= "chooseable-space-overlay" style="background-color: rgba(153, 204, 102, 0.8); cursor: pointer;"
# the background color when an empty tile is highlighted 
# class="space-placeholder" style="background-color: rgba(153, 204, 102, 0.8); cursor: pointer;
# the background color when a tile is not highlighted
# style="background-color: rgba(0, 0, 0, 0); cursor: default;
# the buttons on the left side, go to next phase, etc.
# button class="control-panel-content-actions-action"
# empty spaces 
# class="space-placeholder-container"
# cards that opponent controls when you can force them
# card zoomable highlight mappy opponent-controlled

In [2]:
# !pip install webdriver_manager

In [27]:
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains
np.__version__

'1.19.5'

In [4]:

df = pd.read_excel('../sw_alt_password.xlsx')

In [5]:
password = df['password'][1]
email = df['Email'][1]

In [6]:
factions = ['Breakers', 'Cave Goblins', 'Cloaks', 'Crimson Order', 'Eternal Council', 'Fallen Kingdom'
           , 'Fungal Dwarves', 'High Elves', 'Mountain Vargath', 'Obsidian Dwarves',
           'Phoenix Elves', 'Polar Dwarves', 'Sand Goblins', 'Savanna Elves',
           'Shadow Elves', 'Skyspear Avians', 'Swamp Orcs', 'Tundra Orcs',
           'Vanguards', 'Wayfarers']

In [120]:



def start_game(singleplayer, faction_index, opposing_index, email, password, driver):
    url = 'https://www.plaidhatgames.com/accounts/login/?next=/redirect-swo'
    driver.get(url)



    time.sleep(5)
    login = driver.find_element_by_id('id_login')
    login.click()
    login.send_keys(email)
    time.sleep(.5)
    pw = driver.find_element_by_id('id_password')
    pw.click()
    pw.send_keys(password)
    time.sleep(.05)
    button = driver.find_element_by_class_name('primaryAction')
    button.click()
    time.sleep(15)
    buttons = driver.find_elements_by_class_name('menu-button-text-container')
    if singleplayer==True:
        buttons[1].click()
        time.sleep(3)
        # change some settings
        settings = driver.find_element_by_class_name('settings-button-left')
        settings.click()
        time.sleep(1)
        # auto dismiss events (can cause issues otherwise)
        autodismiss_events = driver.find_element_by_id('autodismiss_events')
        autodismiss_events.click()
        time.sleep(.05)
        # turn on attack friendly
        attack_friendly = driver.find_element_by_id('attack_friendly')
        attack_friendly.click()
        time.sleep(.05)
        # turn off the sound
        sound = driver.find_elements_by_id('text-size-input')
        move = ActionChains(driver)
        move.click_and_hold(sound[0]).move_by_offset(-100, 0).release().perform()
        time.sleep(.05)
        move.click_and_hold(sound[1]).move_by_offset(-100, 0).release().perform()
        time.sleep(.05)

        back_button = driver.find_element_by_class_name('full-overlay-back-button-img')
        back_button.click()
        time.sleep(.5)
        buttons = driver.find_elements_by_class_name('menu-button-text-container')
        buttons[3].click()
        time.sleep(3)
        buttons = driver.find_elements_by_class_name('use-deck-button')
        buttons[faction_index].click()
        time.sleep(1)
        buttons = driver.find_elements_by_class_name('use-deck-button')
        buttons[opposing_index].click()
    else:
        buttons[0].click()



    

In [8]:
# function that determines whether it's your turn
def is_turn(driver):
    time.sleep(.5)
    try:
        driver.find_element_by_class_name('control-panel-content-actions-action')
        return True
    except:
        return False

In [121]:
# driver = webdriver.Chrome(executable_path=r"C:\Users\steve\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(executable_path=r"C:\\Users\\steve\\.wdm\\drivers\\chromedriver\\win64\\119.0.6045.105\\chromedriver.exe")
# use this when you need a new version of chromedriver
# driver = webdriver.Chrome(ChromeDriverManager().install())
start_game(singleplayer=True, faction_index=factions.index('Tundra Orcs'), opposing_index=factions.index('Polar Dwarves'),
          email=email, password=password, driver=driver)
print(is_turn(driver))

False


In [ ]:
# driver.page_source

In [124]:
# returns outer html for each space on the map
def get_board_spaces(driver):
    all_spaces = []
    for col in range(6):
        single_col_spaces = []
        for row in range(8):
            space_html = driver.find_element_by_id(f'map-space-{col}-{row}').get_attribute('outerHTML')
            if 'space-placeholder' in space_html:
                friendly = 0
                occupied = False
                card_abilities = ['']
                card_classes = ['']
            else:
                occupied = True
                
                if 'opponent' in space_html:
                    friendly = -1
            
                else:
                    friendly = 1
                    
                card = driver.find_element_by_id(f'map-space-{col}-{row}')
                card.click()
                time.sleep(.05)
                               
    
                zoom_html = driver.find_element_by_class_name('zoom-card').get_attribute('outerHTML')
        
                
                
                
                if 'zoom-card-ability' in zoom_html:         
                    html_card_abilities = re.findall('<div class="zoom-card-ability-name">[^<]+</div>',zoom_html)
                    regex_card_abilities = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-ability-name">'), re.escape('</div>')))
                    card_abilities = [regex_card_abilities.findall(html_card_abilities[i])[0] for i in range(len(html_card_abilities))]
                    
                    html_card_classes = re.findall('<div class="zoom-card-extras-unit">[^<]+</div>',zoom_html)
                    regex_card_classes = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-extras-unit">'), re.escape('</div>')))
                    card_classes = regex_card_classes.findall(html_card_classes[0])
#                     card_classes = card_classes[0]
                    
                    
                    if 'Structure' in zoom_html:
                        card_classes.append('Structure')
                    print(card_classes)
                    
                    
                else:
                    card_abilities = ['']
                    card_classes = ['Structure']
                    print(card_classes)

                elem = driver.find_element_by_id('zoom-card')
                ac = ActionChains(driver)
                ac.move_to_element(elem).move_by_offset(300, 400).click().perform()
                time.sleep(.05)
                               
            
            
            # card-name
            if 'card-name' in space_html:
                html_card_name = re.findall('alt=[^<]+><div class="card-name">',space_html)
                regex_card_name = re.compile('{}(.*){}'.format(re.escape('alt='), re.escape('><div class="card-name">')))
                card_name = regex_card_name.findall(html_card_name[0])
                card_name = card_name[0]
            else:
                card_name = ''
            
            # card-strength
            if 'card-strength' in space_html:
                html_card_strength = re.findall('<div class="card-strength">[^<]+</div>',space_html)
                regex_card_strength = re.compile('{}(.*){}'.format(re.escape('<div class="card-strength">'), re.escape('</div>')))
                card_strength = regex_card_strength.findall(html_card_strength[0])
                card_strength = card_strength[0]
            else:
                card_strength=0
            
            # card-life
            if 'card-life' in space_html:
                html_card_life = re.findall('<div class="card-life[^<]+</div>',space_html)
                regex_card_life = re.compile('{}(.*){}'.format(re.escape('>'), re.escape('</div>')))
                card_life = regex_card_life.findall(html_card_life[0])
                card_life = card_life[0]
            else:
                card_life=0
            # boost
            if 'boost' in space_html:
                html_card_boost = re.findall('<div class="boost">[^<]+</div>',space_html)
                regex_card_boost = re.compile('{}(.*){}'.format(re.escape('<div class="boost">'), re.escape('</div>')))
                boost = regex_card_boost.findall(html_card_boost[0])
                boost = boost[0]
            else:
                boost=0
            # card-abilities-ability
#             if 'card-abilities-ability' in space_html:
#                 html_card_abilities = re.findall('<div class="card-abilities-ability">[^<]+</div>',space_html)
#                 regex_card_abilities = re.compile('{}(.*){}'.format(re.escape('<div class="card-abilities-ability">'), re.escape('</div>')))
#                 card_abilities = [regex_card_abilities.findall(html_card_abilities[i])[0] for i in range(len(html_card_abilities))]
#             else:
#                 card_abilities = ['']
            # append to the column list
            single_col_spaces.append([occupied, friendly, card_name, card_strength, card_life, boost, card_abilities,
                                      card_classes, row, col])
        
        # append to columns to all_spaces so it is a 2 dimensional list (can maybe perform convolutions?)
        all_spaces.append(single_col_spaces)
            
    return all_spaces        
            

In [125]:
spaces = get_board_spaces(driver)
spaces

['Summoner']
['Common', 'Structure']
['Structure']
['Common', 'Structure']
['Common']
['Summoner']
['Common']
['Common']
['Structure']


[[[False, 0, '', 0, 0, 0, [''], [''], 0, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 1, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 2, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 3, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 4, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 5, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 6, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 7, 0]],
 [[False, 0, '', 0, 0, 0, [''], [''], 0, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 1, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 2, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 3, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 4, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 5, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 6, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 7, 1]],
 [[True,
   -1,
   '"Svara"',
   '3',
   '12',
   0,
   ['Structural Shift'],
   ['Summoner'],
   0,
   2],
  [True,
   -1,
   '"Ice Golem"',
   '2',
   '5',
   0,
   ['Living Gate', 'Mobile Structure', 'Slow'],
   ['Common', 'Structure'],
   1,
   2],
  [True, -1, '"gate"', 0,

In [86]:
def get_active_events(driver):
    try:
        active_html = driver.find_element_by_class_name('active-display-opponent').get_attribute('outerHTML')
        html_active_events = re.findall('title=[^=]+',active_html)
        regex_active_events = re.compile('{}(.*){}'.format(re.escape('title="'), re.escape('" class')))
        opponent_active_events = [regex_active_events.findall(html_active_events[i])[0] for i in range(len(html_active_events))]
    except:
        opponent_active_events = ['']
    try:
        active_html = driver.find_element_by_class_name('active-display-mine').get_attribute('outerHTML')
        html_active_events = re.findall('title=[^=]+',active_html)
        regex_active_events = re.compile('{}(.*){}'.format(re.escape('title="'), re.escape('" class')))
        friendly_active_events = [regex_active_events.findall(html_active_events[i])[0] for i in range(len(html_active_events))]
    except:
        friendly_active_events = ['']
    return opponent_active_events, friendly_active_events

In [88]:
opponent_active_events, friendly_active_events = get_active_events(driver)
opponent_active_events, friendly_active_events

([], ['Insight', 'Learn'])

In [122]:
# function to obtain amount of magic, cards in hand and deck
def get_resources(driver):
    user_frames = driver.find_elements_by_class_name('user-framed-stuff')
    enemy_frames = user_frames[0].get_attribute('outerHTML')
    html_magic = re.findall('title="magic"><span>[^<]+</span>',enemy_frames)
    regex_magic = re.compile('{}(.*){}'.format(re.escape('title="magic"><span>'), re.escape('</span>')))
    enemy_magic = regex_magic.findall(html_magic[0])
    enemy_magic = enemy_magic[0]
    html_hand = re.findall('title="hand"><span>[^<]+</span>',enemy_frames)
    regex_hand = re.compile('{}(.*){}'.format(re.escape('title="hand"><span>'), re.escape('</span>')))
    enemy_hand = regex_hand.findall(html_hand[0])
    enemy_hand = enemy_hand[0]
    html_draw = re.findall('title="draw"><span>[^<]+</span>',enemy_frames)
    regex_draw = re.compile('{}(.*){}'.format(re.escape('title="draw"><span>'), re.escape('</span>')))
    enemy_draw = regex_draw.findall(html_draw[0])
    enemy_draw = enemy_draw[0]
    
    friendly_frames = user_frames[1].get_attribute('outerHTML')
    html_magic = re.findall('title="magic"><span>[^<]+</span>',friendly_frames)
    regex_magic = re.compile('{}(.*){}'.format(re.escape('title="magic"><span>'), re.escape('</span>')))
    friendly_magic = regex_magic.findall(html_magic[0])
    friendly_magic = friendly_magic[0]
    html_hand = re.findall('title="hand"><span>[^<]+</span>',friendly_frames)
    regex_hand = re.compile('{}(.*){}'.format(re.escape('title="hand"><span>'), re.escape('</span>')))
    friendly_hand = regex_hand.findall(html_hand[0])
    friendly_hand = friendly_hand[0]
    html_draw = re.findall('title="draw"><span>[^<]+</span>',friendly_frames)
    regex_draw = re.compile('{}(.*){}'.format(re.escape('title="draw"><span>'), re.escape('</span>')))
    friendly_draw = regex_draw.findall(html_draw[0])
    friendly_draw = friendly_draw[0]
    return [int(enemy_magic), int(enemy_hand), int(enemy_draw), int(friendly_magic), int(friendly_hand), int(friendly_draw)]


In [123]:
resources = get_resources(driver)
resources

[3, 5, 21, 3, 5, 25]

In [ ]:
def get_hand(driver):

In [ ]:
# for multiplayer where you don't get to choose opponent
def get_enemy_summoner(driver):

In [ ]:
# returns list of card names, abilities, units/structures(cards that can go on battlefield), active events, 
# cards that go under other cards (will be used to create a list of these for one hot encoding)
def list_card_names_types_and_abilities(driver):

In [ ]:
def get_discard(driver):

In [ ]:
def get_deck(driver):

In [ ]:
def get_enemy_discards_remaining_cards(driver)

In [ ]:
unit_classes = ['Summoner', 'Structure', 'Common', 'Champion']

In [ ]:
# observation, reward, done, and info?

In [ ]:
# gets the current state of the board, cards in hand, discarded cards, active events, cards in deck, etc.
def get_board_state()

In [ ]:
# possible actions that can be taken at any given point
def get_available_actions()

In [ ]:
# choose an action, evaluating them?? (need to refresh on reinforcement learning here)
def evaluate_actions()